#### Load Path, Libraries, and Modules

In [1]:
import os
import sys
import pandas as pd

# Define the correct path to scr
module_path = os.path.abspath(os.path.join("..", "scripts")) 

# Add path only if it exists
if module_path not in sys.path and os.path.exists(module_path):
    sys.path.append(module_path)

# Now import the scraper module
from scraper import AppReviewsScraper

### Scraping and Storing Dataset

In [2]:
# Define app IDs
app_ids = {
    "CBE": "com.combanketh.mobilebanking",
    "Dashin": "com.dashen.dashensuperapp",
    "Abyissinia": "com.boa.boaMobileBanking"
    }

# Instantiate and use the scraper
scraper = AppReviewsScraper(app_ids)
df_combined = scraper.get_reviews_dataframe()

# Display the resulting DataFrame
print(df_combined.head())

                               reviewId       userName  \
0  be2cb2ac-bbe0-4175-81c4-9f6c86afdaaa  A Google user   
1  8efd71e9-59cd-41ce-8c5c-12052dee9ad0  A Google user   
2  b12d0383-9b27-4e49-a94d-277a43b15800  A Google user   
3  dd9f9e37-177a-46df-b877-d0edaa9aed29  A Google user   
4  8e34703c-203c-4180-8b32-bfd0b3f0c871  A Google user   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0  "Why don’t your ATMs support account-to-accoun...      4              0   
1                        what is this app problem???      1              0   
2       the app is proactive and a good connections.      5              0   
3 

c:\Users\girma\OneDrive\Desktop\10 Academy\week-2\bank-app-reviews\scripts\scraper.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(dataframes, ignore_index=True)


In [3]:
#save the dataset
df_combined.to_csv("../data/raw_reviews.csv", index=False)

### Preprocessing - Data Cleaning

In [4]:
# consise summary of the dataframe
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8991 entries, 0 to 8990
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              8991 non-null   object        
 1   userName              8991 non-null   object        
 2   userImage             8991 non-null   object        
 3   content               8984 non-null   object        
 4   score                 8991 non-null   int64         
 5   thumbsUpCount         8991 non-null   int64         
 6   reviewCreatedVersion  7263 non-null   object        
 7   at                    8991 non-null   datetime64[ns]
 8   replyContent          70 non-null     object        
 9   repliedAt             70 non-null     datetime64[ns]
 10  appVersion            7263 non-null   object        
 11  App                   8991 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(8)
memory usage: 843.0+ KB


In [5]:
# add source of the dataset column 
df_combined["source"] = "Google Play"

# rename desired columns to appropriate names.
df_combined = df_combined.rename(columns={    
    "content": "review",
    "score": "rating",
    "at": "date",
    "App": "bank",
})

# Ensure the dataset has these columns
df_combined.columns

Index(['reviewId', 'userName', 'userImage', 'review', 'rating',
       'thumbsUpCount', 'reviewCreatedVersion', 'date', 'replyContent',
       'repliedAt', 'appVersion', 'bank', 'source'],
      dtype='object')

In [6]:
# Remove duplicates
df_cleaned = df_combined.drop_duplicates()

In [7]:
#check missing values
df_cleaned.isnull().sum()

reviewId                   0
userName                   0
userImage                  0
review                     7
rating                     0
thumbsUpCount              0
reviewCreatedVersion    1728
date                       0
replyContent            8921
repliedAt               8921
appVersion              1728
bank                       0
source                     0
dtype: int64

`The review contains seven missing entries, and dropping these rows is reasonable since the content is already absent.`

In [8]:
print(f"Missing Reviews Before Cleanup: {df_cleaned['review'].isnull().sum()}")

Missing Reviews Before Cleanup: 7


In [9]:
# drop missing values
df_cleaned.dropna(subset=['review'], inplace=True)

In [10]:
print(f"Missing Reviews After Cleanup: {df_cleaned['review'].isnull().sum()}")

Missing Reviews After Cleanup: 0


In [14]:
# Normalize Dates (YYYY-MM-DD
df_cleaned["date"] = pd.to_datetime(df_cleaned["date"]).dt.strftime("%Y-%m-%d")

In [15]:
# choose the desired columns
df_cleaned = df_cleaned[["bank", "review", "rating", "date", "source"]]

In [16]:
#check data types after cleaning
print(df_cleaned.dtypes)

bank      object
review    object
rating     int64
date      object
source    object
dtype: object


In [23]:
# Convert the "date" column to proper datetime format
df_cleaned["date"] = pd.to_datetime(df_cleaned["date"], errors="coerce")

# Check for any rows where conversion failed
invalid_dates = df_cleaned[df_cleaned["date"].isna()]
print(f"Found {len(invalid_dates)} invalid date entries!")

# Drop rows with invalid dates
df_cleaned.dropna(subset=["date"], inplace=True)


# Ensure the data types
df_cleaned.dtypes

Found 0 invalid date entries!


bank              object
review            object
rating             int64
date      datetime64[ns]
source            object
dtype: object

In [24]:
# save cleaned data as csv
df_cleaned.to_csv("../data/cleaned_reviews.csv", index=False)